In [2]:
import os
import random
import numpy as np
import tensorflow
import tensorflow.keras
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [46]:
dataDir = './data/'
countryDirs = os.listdir(dataDir)

data = []

for cdir in countryDirs:
    fileDir = (os.path.join(dataDir, cdir, 'summary.txt'))
    data.append(open(fileDir, 'r').read())
data = ' '.join(data)
print(len(data))

uniqueCharacters = list(set(data)) + ['*', '_']
print(len(uniqueCharacters))
print(uniqueCharacters)

608145
55
['.', '4', 'i', 'e', 'j', '"', 'k', '8', ' ', 'd', 'g', '(', 'l', 'm', ';', '&', 'a', "'", '[', 'b', '5', '6', '+', 't', '$', 'h', 'f', 'c', '1', 'o', 'x', ')', 'p', '0', '%', '/', '3', 'r', 's', ',', 'u', ':', '2', 'q', '7', 'y', 'v', ']', 'n', '-', '9', 'w', 'z', '*', '_']


In [37]:
data = list(text1)# + list(text2) + list(text3) + list(text4) + list(text5) + list(text6) + list(text7) + list(text8) + list(text9)
data = ' '.join(data)
data = data.lower()
data = data.encode('ascii', 'ignore').decode()
uniqueCharacters = list(set(data).union(['*', '_', '+']))
print(len(uniqueCharacters))
print(uniqueCharacters)

55
['.', 'j', 'i', 'e', '"', '4', 'k', '8', ' ', 'd', 'g', '(', 'm', 'l', ';', '&', 'a', "'", '[', 'b', '5', '6', '*', '+', 't', '$', 'h', 'f', 'c', '1', 'o', ')', 'x', 'p', '0', '3', 'r', '!', 's', 'u', ',', ':', 'q', '2', '7', 'y', 'v', ']', '?', '_', 'n', '-', '9', 'w', 'z']


In [47]:
xLength = 30
stepSize = 1

dataX = []
dataY = []

for i in range(0, len(data) - xLength, stepSize):
    dataX.append(data[i:i + xLength])
    dataY.append(data[i + xLength])
print(len(dataX))

608115


In [48]:
def vectorize(X, xLength):
    res = np.zeros((len(X), xLength, len(uniqueCharacters)))
    for N, i in enumerate(X):
        for n, x in enumerate(i):
            idx = uniqueCharacters.index(x)
            res[N, n, idx] = 1
    return res
dataX = vectorize(dataX, xLength)
# dataY = vectorize(dataY, 1)
dataY = np.squeeze(vectorize(dataY, 1), axis = 1)
print(dataX.shape)
print(dataY.shape)

(608115, 30, 55)
(608115, 55)


In [40]:
def rev_vectorize(x):
    x = np.squeeze(x, axis = 0)
    x = x.tolist()
    max_ = max(x)
    idx = x.index(max_)
    return uniqueCharacters[idx]

In [41]:
model = Sequential()
model.add(LSTM(units = 64, input_shape = (xLength, len(uniqueCharacters))))
model.add(Dense(len(uniqueCharacters), activation='softmax'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                30720     
_________________________________________________________________
dense (Dense)                (None, 55)                3575      
Total params: 34,295
Trainable params: 34,295
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.compile(loss='categorical_crossentropy', optimizer=Adamax(lr = 0.01, decay = 0.0001))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                30720     
_________________________________________________________________
dense (Dense)                (None, 55)                3575      
Total params: 34,295
Trainable params: 34,295
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.fit(dataX, dataY, batch_size = 64, epochs = 5)

Epoch 1/5
608115/608115 [==============================] - 328s 539us/step - loss: 2.1836
Epoch 2/5
608115/608115 [==============================] - 326s 537us/step - loss: 1.5702
Epoch 3/5
608115/608115 [==============================] - 318s 523us/step - loss: 1.4827s - l
Epoch 4/5
608115/608115 [==============================] - 315s 518us/step - loss: 1.4361
Epoch 5/5
608115/608115 [==============================] - 319s 525us/step - loss: 1.4039


In [43]:
model.save(f'./models/model_{xLength}_{stepSize}_text1')

In [51]:
model = load_model(f'./models/model_{xLength}_{stepSize}_text1')

In [44]:
def sample(x, temp = 0.6):
    x = x[0]
    preds = np.log(x) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    maxloc = preds.argmax()
    preds[maxloc] = preds[maxloc] - 0.0001
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def predictText(x, predictLength = 1000):
#     assert len(x) == xLength
    results = []
    for temp in range(1, 10, 2):
        result = x
        input_text = x
        for _ in range(predictLength):
            input_text_vectorized = vectorize([input_text], xLength)
            nextChr = model.predict(input_text_vectorized)
            nextChr = sample(nextChr, temp/10)
            nextChr = uniqueCharacters[nextChr]
            result += nextChr
            input_text += nextChr
            input_text = input_text[1:]
        print(temp/10, result)
        print()
        results.append((temp, result))
    return results

In [53]:
rndPoint = random.randint(0, len(data))
startString = data[rndPoint:rndPoint + xLength]
startString = "kuztistan is a republic in the "
startString = startString[:xLength]
print(startString)
result = predictText(startString, 1000)

kuztistan is a republic in the


/home/shahzaib/anaconda3/envs/t/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


0.1 kuztistan is a republic in the southeast , and the country ' s country of the country ' s country of the country ' s country of the country ' s consisting the southern the country , and the country ' s in the southeast , and the country ' s country and the country ' s country of the country ' s country of the country ' s country , and the country ' s country of the country ' s country of the country ' s country of the country ' s country of the country ' s in the southern in the southeast , and the country ' s country of the country ' s country of the country ' s country of the country ' s country of the country ' s country of the country ' s country of the country ' s country of the country ' s country , and the country ' s country in the world , and the country ' s country of the country ' s country of the country ' s country of the country , and the country ' s country of the country ' s country , and the country ' s country of the country ' s country of the country ' s country 